## Lung Cancer Prediction - Feature Selection

In [23]:
#import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly
import math


In [38]:
#load the data

curr_dir=os.getcwd()
f_path = os.path.join(curr_dir, "data", "cancer patient data sets.csv")

df=pd.read_csv(f_path)
features = df.drop(["index","Patient Id","Level"], axis=1)
features.head()

,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,Obesity,...,Coughing of Blood,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring
0,33,1,2,4,5,4,3,2,2,4,...,4,3,4,2,2,3,1,2,3,4
1,17,1,3,1,5,3,4,2,2,2,...,3,1,3,7,8,6,2,1,7,2
2,35,1,4,5,6,5,5,4,6,7,...,8,8,7,9,2,1,4,6,7,2
3,37,1,7,7,7,7,6,7,7,7,...,8,4,2,3,1,4,5,6,7,5
4,46,1,6,8,7,7,7,6,7,7,...,9,3,2,4,1,4,2,4,2,3


### Feature selection

We will use 3 feature selection to be compred within the baseline SVM model.
- method 1: scikit-learn select_best_k features
- method 2: Principal Component Analysis (PCA)
- method 3: DecisionTree's Feature Importance

In [65]:
# Check accuracy using all features
from sklearn.model_selection import train_test_split
X=features
y=df["Level"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [101]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# A Naive SVM linear model will be used as benchmark of the feature selection.
def baseline_accuracy(feature_selection):
    X_train_subset = X_train[feature_selection]
    X_test_subset = X_test[feature_selection]
    linear_clf = svm.SVC(kernel="linear", decision_function_shape='ovo', probability=True)
    linear_clf.fit(X_train_subset, y_train)
    y_pred = linear_clf.predict(X_test_subset)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: ", accuracy)

baseline_accuracy(features.columns)


Accuracy:  1.0


The baseline accuracy without any feature selection is 1. The following section will explore selections of features to see if it affects the accuracy.

#### Method 1: scikit-learn select_best_k features

In [102]:
from sklearn.feature_selection import SelectKBest
X_train_2 , X_test_2, y_train_2, y_test_2=X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()
select_k_best=SelectKBest()
x_col=select_k_best.fit_transform(X_train_2,y_train_2)
features_selected_method1=X_train_2.columns[select_k_best.get_support()]
print("Features:")
print(features_selected_method1)
baseline_accuracy(features_selected_method1)


Features:
Index(['Air Pollution', 'Alcohol use', 'Dust Allergy', 'OccuPational Hazards',
       'Genetic Risk', 'Balanced Diet', 'Obesity', 'Passive Smoker',
       'Chest Pain', 'Coughing of Blood'],
      dtype='object')
Accuracy:  0.925


SelectKBest selects features using a scoting method. It defaults to k-10, and the method for categorical data is using Chi-square tests. The limited number of k probably explains why the accuracy goes down compared to the baseline.

#### Method 2: Principal Component Analysis (PCA)

The features that contributes to the highest variance, indicated by the principal components (PC) will be extracted.
- Method 2a: Uses 2 PC
- Method 2b: Uses 3 PC

In [94]:
x = features
y=df["Level"]
n_comp=10
pc_list = [f"PC{i+1}" for i in range(n_comp)]

from sklearn import decomposition
pca = decomposition.PCA(n_components=n_comp)
pc = pca.fit_transform(x)
pc_df = pd.DataFrame(data=pc,columns=pc_list)
pc_df['Label'] = y

var = pd.DataFrame({'Variance':pca.explained_variance_ratio_,
             'PC':pc_list})

#Make a 3D plot for the PC
import plotly.express as plotly_express
figure = plotly_express.scatter_3d(pc_df,x="PC1",y="PC2",z="PC3",color="Label",)
figure.update_layout(showlegend=True,
                     scene=dict(
                         xaxis_title="PC1 ("+ str(round(var['Variance'][0]*100,2)) + "%)",
                         yaxis_title="PC2 ("+ str(round(var['Variance'][1]*100,2)) + "%)",
                         zaxis_title="PC3 ("+ str(round(var['Variance'][2]*100,2)) + "%)"
                     ))
figure.show()


From the 3d plot, the 1st, 2nd, and 3rd PC contributes to 59.54%, 18.00% and 5.49% of variance respectively.
Even from using PC1 and PC2 alone, it accounts for 77.6% of the variance.

Trying out taking the most important features from 2 PCs

In [79]:
n_pcs = 2

model = decomposition.PCA(n_components=n_pcs).fit(X_train)
f_ordered = [np.argsort(model.components_[i]) for i in range(n_pcs)]
feature_names = features.columns
f = []
for i in range(n_pcs):
    x = [feature_names[j] for j in f_ordered[i]]
    f.append(x)
# 5 most important features in each PC 
# get the last 5 features (features sorted in increasing order of importance)
# reverse list to sort by most important 
ls = [f[i][-5:][::-1] for i in range(n_pcs)]
data = [['PC{}'.format(i)]+ls[i] for i in range(n_pcs)]
df_features = pd.DataFrame(data,columns=["PC",1,2,3,4,5])
final_features = ls[0] + ls[1]
final_features_method2a = list(dict.fromkeys(final_features))
print("Features:")
print(final_features_method2a)
baseline_accuracy(final_features_method2a)

Features:
['Age', 'Alcohol use', 'Smoking', 'chronic Lung Disease', 'Fatigue', 'Coughing of Blood', 'Chest Pain', 'Genetic Risk', 'OccuPational Hazards']
Accuracy:  0.945


Trying out taking the most important features from 2 PCs

In [78]:
n_pcs = 3

model = decomposition.PCA(n_components=n_pcs).fit(X_train)
f_ordered = [np.argsort(model.components_[i]) for i in range(n_pcs)]
feature_names = features.columns
f = []
for i in range(n_pcs):
    x = [feature_names[j] for j in f_ordered[i]]
    f.append(x)

# 5 most important features in each PC 
# get the last 5 features (features sorted in increasing order of importance)
# reverse list to sort by most important 
ls = [f[i][-5:][::-1] for i in range(n_pcs)]
data = [['PC{}'.format(i)]+ls[i] for i in range(n_pcs)]
df_features = pd.DataFrame(data,columns=["PC",1,2,3,4,5])
final_features = ls[0] + ls[1] + ls[2]
final_features_method2b = list(dict.fromkeys(final_features))
print("Features:")
print(final_features_method2b)
baseline_accuracy(final_features_method2b)

Features:
['Age', 'Alcohol use', 'Smoking', 'chronic Lung Disease', 'Fatigue', 'Coughing of Blood', 'Chest Pain', 'Genetic Risk', 'OccuPational Hazards', 'Shortness of Breath', 'Weight Loss', 'Clubbing of Finger Nails', 'Dry Cough', 'Wheezing']
Accuracy:  1.0


Looking at the PCA plot, PC1 is unable to show separation between the 3 classes. This meaning most variance captured by PC1 are mostly variance within the classes.
However, PC2 and PC3 are able to capture the class separation better.

This probably explains why using only 2 PC for the feature selection doesn't yield as good accuracy as using 3 PC.

#### Method 3: DecisionTree's Feature Importance

In [69]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

# Train decision tree classifier
clf.fit(X_train, y_train)

# Get feature importances
feature_importances = clf.feature_importances_

# Create a dictionary mapping feature names to importances
feature_importance_dict = dict(zip(list(X_train.columns), feature_importances))

# Sort features by importance
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

print("Feature Importances:")
for feature, importance in sorted_features:
    print(f"{feature}: {importance:.4f}")

Feature Importances:
Coughing of Blood: 0.3734
Wheezing: 0.2804
Obesity: 0.1290
Snoring: 0.1131
OccuPational Hazards: 0.0713
Clubbing of Finger Nails: 0.0327
Age: 0.0000
Gender: 0.0000
Air Pollution: 0.0000
Alcohol use: 0.0000
Dust Allergy: 0.0000
Genetic Risk: 0.0000
chronic Lung Disease: 0.0000
Balanced Diet: 0.0000
Smoking: 0.0000
Passive Smoker: 0.0000
Chest Pain: 0.0000
Fatigue: 0.0000
Weight Loss: 0.0000
Shortness of Breath: 0.0000
Swallowing Difficulty: 0.0000
Frequent Cold: 0.0000
Dry Cough: 0.0000


In [77]:
features_selected_method3 = ['Coughing of Blood', 'Wheezing', 'Obesity', 'Snoring', 'OccuPational Hazards', 'Clubbing of Finger Nails']
print("Features:")
print(features_selected_method3)
baseline_accuracy(features_selected_method3)

Features:
['Coughing of Blood', 'Wheezing', 'Obesity', 'Snoring', 'OccuPational Hazards', 'Clubbing of Finger Nails']
Accuracy:  0.995


#### Features selection results

From the 4 accuracies obtained from the different methods are as follows.
- Baseline: 1.000

- Method 1: 0.925 (SelectKBest)
- Method 2a: 0.945 (2 PC)
- Method 2b: 1.000 (3 PC)
- Method 3: 0.995 (DecisionTree's Feature Importance)

Thus, taking the selection method that yields the least amount of features but maximixing accuracy, features from method 2b will be used.

In [105]:
final_features_method2b

['Age',
 'Alcohol use',
 'Smoking',
 'chronic Lung Disease',
 'Fatigue',
 'Coughing of Blood',
 'Chest Pain',
 'Genetic Risk',
 'OccuPational Hazards',
 'Shortness of Breath',
 'Weight Loss',
 'Clubbing of Finger Nails',
 'Dry Cough',
 'Wheezing']